In [ ]:
!pip install --quiet transformers evaluate datasets baal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 30.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.6/419.6 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install --quiet --upgrade accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 5.8 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/RuSportSum/train.csv" /content/train.csv
!cp "/content/drive/MyDrive/RuSportSum/eval.csv" /content/eval.csv

#### Разметка релевантных новостей с активным обучением

In [ ]:
import os
import random
from copy import deepcopy

import numpy as np
import torch.backends
import transformers
import evaluate

from datasets import load_dataset
from tqdm import tqdm
from transformers import BertForSequenceClassification
from transformers import BertTokenizer, TrainingArguments
from transformers import set_seed

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    set_seed,
    Trainer,
    TrainingArguments
)

transformers.utils.logging.set_verbosity_warning()

from baal.active import get_heuristic
from baal.active.dataset.nlp_datasets import (
    active_huggingface_dataset,
    HuggingFaceDatasets,
)
from baal.bayesian.dropout import patch_module
from baal.transformers_trainer_wrapper import BaalTransformersTrainer
from baal.active import get_heuristic, ActiveLearningDataset
from baal.active.dataset.base import Dataset

from typing import List
from pprint import pprint

SEED = 3

random.seed(SEED)
torch.manual_seed(SEED)

# Set tranformer seed to ensure that initial weights are identical
set_seed(SEED)

In [ ]:
class CustomHuggingFaceDatasets(Dataset):
    """
    Support for `huggingface.datasets`: (https://github.com/huggingface/datasets).
    The purpose of this wrapper is to separate the labels from the rest of the sample information
    and make the dataset ready to be used by `baal.active.ActiveLearningDataset`.
    Args:
        dataset (Dataset): a dataset provided by huggingface.
        tokenizer (transformers.PreTrainedTokenizer): a tokenizer provided by huggingface.
        target_key (str): target key used in the dataset's dictionary.
        input_key (str): input key used in the dataset's dictionary.
        max_seq_len (int): max length of a sequence to be used for padding the shorter
            sequences.
    """

    def __init__(
        self,
        dataset,
        tokenizer=None,
        target_key: str = "label",
        input_key_1: str = "sentence1",
        input_key_2: str = "sentence1",
        max_seq_len: int = 400,
    ):
        self.dataset = dataset
        self.targets, self.text1, self.text2 = self.dataset[target_key], self.dataset[input_key_1], self.dataset[input_key_2]
        self.targets_list: List = np.unique(self.targets).tolist()
        self.input_ids, self.attention_masks = (
            self._tokenize(tokenizer, max_seq_len) if tokenizer else ([], [])
        )

    @property
    def num_classes(self):
        return len(self.targets_list)

    def _tokenize(self, tokenizer, max_seq_len):
        # For speed purposes, we should use fast tokenizers here, but that is up to the caller
        tokenized = tokenizer(
            self.text1,
            self.text2,
            add_special_tokens=True,
            max_length=max_seq_len,
            return_token_type_ids=False,
            padding="max_length",
            return_attention_mask=True,
            return_tensors="pt",
            truncation=True,
        )
        return tokenized["input_ids"], tokenized["attention_mask"]

    def label(self, idx: int, value: int):
        """Label the item.
        Args:
            idx: index to label
            value: Value to label the index.
        """
        self.targets[idx] = value

    def __len__(self):
        return len(self.text1)

    def __getitem__(self, idx):
        target = self.targets_list.index(self.targets[idx])

        return {
            "input_ids": self.input_ids[idx].flatten() if len(self.input_ids) > 0 else None,
            "inputs": self.text1[idx] + '\n' + self.text2[idx],
            "attention_mask": self.attention_masks[idx].flatten()
            if len(self.attention_masks) > 0
            else None,
            "label": torch.tensor(target, dtype=torch.long),
        }

def custom_active_huggingface_dataset(
    dataset,
    tokenizer=None,
    target_key: str = "label",
    input_key_1: str = "sentence1",
    input_key_2: str = "sentence2",
    max_seq_len: int = 400,
    **kwargs
):
    """
    Wrapping huggingface.datasets with baal.active.ActiveLearningDataset.
    Args:
        dataset (torch.utils.data.Dataset): a dataset provided by huggingface.
        tokenizer (transformers.PreTrainedTokenizer): a tokenizer provided by huggingface.
        target_key (str): target key used in the dataset's dictionary.
        input_key (str): input key used in the dataset's dictionary.
        max_seq_len (int): max length of a sequence to be used for padding the shorter sequences.
        kwargs (Dict): Parameters forwarded to 'ActiveLearningDataset'.
    Returns:
        an baal.active.ActiveLearningDataset object.
    """

    return ActiveLearningDataset(
        CustomHuggingFaceDatasets(dataset, tokenizer, target_key, input_key_1, input_key_2, max_seq_len), **kwargs
    )

Information on the hyperparms below

* epoch: Number of times you want to run and AL loop
* batch_size: The train and eval batch size for hf trainer arguments
* model: Hugging Face Model
* query_size: Number of samples you want to query at each AL iteration for labelling
* heuristic: The acquisition function/heuristic based on which you want to query the important samples
* iterations: The number of iterations you want to run for MCdropout to find the uncertanities
* shuffle_prop: Additional Noise to counter selection bias
* learning_epoch: Traing epochs for hugging face trainer

In [ ]:
hyperparams = {
    "epoch": 4,
    "batch_size": 16,
    "model": "DeepPavlov/rubert-base-cased",
    "query_size": 50,
    "heuristic": "bald",
    "iterations": 10,
    "shuffle_prop": 0.05,
    "learning_epoch": 2,
}

In [ ]:
# Check for CUDA
use_cuda = torch.cuda.is_available()
torch.backends.cudnn.benchmark = True

id2label = {0: "LABEL_0", 1: "LABEL_1"}
label2id = {"LABEL_0": 0, "LABEL_1": 1}

# Load Model
hf_model = AutoModelForSequenceClassification.from_pretrained(
    hyperparams["model"], num_labels=2, id2label=id2label, label2id=label2id
    )

# Setup tokenizer for model
tokenizer = AutoTokenizer.from_pretrained(hyperparams["model"])

# Enable dropouts for predictions
hf_model = patch_module(hf_model)

# Send model to device and setup cuda arguments
if use_cuda:
    hf_model.to("cuda:0")
    no_cuda = False
else:
    hf_model.to("cpu")
    no_cuda = True

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

In [ ]:
# Define labels in your dataset
label_list = [0, 1]

# Load data from files
data = load_dataset("csv", data_files={'train': '/content/train.csv', 'eval': "/content/eval.csv"})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-5c97641fdcada1ed/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def get_label_from_data(active_dataset, data, target, input1, input2, indexes) -> List[int]:
    """
    Get labels from the active dataset, this assumes that you have
    already labelled some samples in your initial dataset

    Args:
    ----
    active_dataset : Active dataset which consists of train and pool

    indexes : Indexes of the points for which labels are to be fetched
    from the data

    Returns:
    ----
    labels: Returns the corresponding labels

    """

    labels = []

    # Now since you labelled points earlier now some part of pool has become train
    # so in order to get the pool indexes based on your 'original' data i.e
    # your raw_train_set. Make sure to user __pool_tp

    raw_data_idx = active_dataset._pool_to_oracle_index(indexes)

    for idx in raw_data_idx:
        #print(f"Adding labels for Raw data Index {idx} : {data[input1][idx]}")
        #print(f"Adding labels for Raw data Index {idx} : {data[input2][idx]}")

        label = data[target][idx]
        labels.append(label)
        #print(label)
        #print("\n")

    return labels

In [ ]:
def get_label_human_oracle(active_dataset, indexes) -> List[int]:
    """
    Get labels from human oracle. During the AL loop some samples
    will go to the human labeller

    Args:
    ----
    active_dataset : Active dataset which consists of train and pool

    indexes : Indexes of the points for which labels are to be fetched
    from the data

    Returns:
    ----
    labels: Returns the corresponding labels

    """
    # List for corresponding labels
    labels = []

    skipped = []

    for sample_idx, idx in enumerate(indexes):

        while True:
            try:
                print(idx)
                pprint(active_dataset.pool.__getitem__(idx)['inputs'], width=150)
                label = int(input())
            except ValueError:
                print("Sorry, I didn't understand that.")
                continue
            if label != -1 and label not in label_list:
                print(f"Allowed labels are {label_list}")
                continue
            if label == -1:
                print("Skipping this sample")
                skipped.append(sample_idx)
                break
            else:
                labels.append(label)
                break
        print("\n")

    indexes_upd = np.delete(indexes, skipped)

    return labels, indexes_upd

In [ ]:
#small_data = data['train'].select(range(10_000))
small_data = data['train']

active_set = custom_active_huggingface_dataset(
    small_data,
    tokenizer=tokenizer,
    target_key='result_match',
    input_key_1='name_body',
    input_key_2='last_part_content'
)

active_set.can_label = True

# 700 первых объектов набора данных
label_from_data = get_label_from_data(active_set, small_data, 'result_match', 'name_body', 'last_part_content', range(700))

active_set.label(
    range(700),
    label_from_data,
)

valid_set = CustomHuggingFaceDatasets(data['eval'], tokenizer=tokenizer, target_key='result_match', input_key_1='name_body', input_key_2='last_part_content')

active_set, test_set = active_set, valid_set

In [ ]:
# Setup Heuristics
heuristic = get_heuristic(
    hyperparams["heuristic"], hyperparams["shuffle_prop"]
)

# Model save checkpoint
save_checkpoint = 2

metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return metric.compute(predictions=predictions, references=labels)

# Keep track of initial model weights
init_weights = deepcopy(hf_model.state_dict())

training_args = TrainingArguments(
    output_dir=".",
    num_train_epochs=hyperparams["learning_epoch"],
    per_device_train_batch_size=hyperparams["batch_size"],
    per_device_eval_batch_size=hyperparams["batch_size"],
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

# Active Learning Trainer Wrapper
baal_trainer = BaalTransformersTrainer(
    model=hf_model,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)

In [ ]:
logs = []
last_epoch = hyperparams["epoch"] - 1

for epoch in tqdm(range(hyperparams["epoch"])):
    # we use the default setup of HuggingFace for training (ex: epoch=1).
    # The setup is adjustable when BaalHuggingFaceTrainer is defined.
    baal_trainer.train()
    print("\n")

    # Validation!
    #eval_metrics = baal_trainer.evaluate()
    #print("\n")

    if (epoch != last_epoch):
      # MCdropout to gather uncertanities
      predictions = baal_trainer.predict_on_dataset(
          active_set.pool, iterations=hyperparams["iterations"]
      )
      print("\n")

      # Acquistion of the most informative and diverse samples based on BatchBALD
      top_uncertainty = heuristic(predictions)[: hyperparams.get("query_size", 1)]

      # Send the samples for labelling from human oracle
      label_from_oracle, points_to_label_oracle = get_label_human_oracle(
          active_set, top_uncertainty
      )

      # Label active dataset
      active_set.label(points_to_label_oracle, label_from_oracle)

      # We reset the model weights to relearn from the new trainset.
      baal_trainer.load_state_dict(init_weights)
      baal_trainer.lr_scheduler = None

      active_logs = {
          "epoch": epoch,
          "labeled_data": active_set.labelled_map,
          "Next Training set size": len(active_set),
      }
      logs.append({**active_logs})

  0%|          | 0/4 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,0.449400,0.531894,0.796209
2,0.209000,0.241835,0.936759




[275-MainThread  ] [baal.transformers_trainer_wrapper:predict_on_dataset_generator:67] 2023-05-28T10:48:54.829879Z [info     ] Start Predict                  dataset=11764



100%|██████████| 736/736 [52:52<00:00,  4.31s/it]




7266
('Бразилия – Колумбия. Неймар покинул поле на носилках. Нападающий сборной Бразилии Неймар получил травму в матче 1/4 финала чемпионата мира-2014. '
 'Форвард «Барселоны» покинул поле на носилках после удара коленом в спину от колумбийского футболиста. Вместо него на поле вышел Энрике. Бразилия – '
 'Колумбия – 2:1. Как это было\n'
 'Рамос не хуже разных неймаров раскидал соперников на бегу финтами и вырулил атаку своей сборной на финиш 4 в 4! Однако пас в офсайд запортил все '
 'дело. Паулиньо, уходя с поля, показывает, кого не надо забывать держать. Эрнанес вместо него. Суньига с разбега коленом в крестец - приличный '
 'рестлеровский прием выводит Неймара из игры. Энрике вместо Неймара, который на носилках транспортируется прямиком в раздевалку. Блокируют удар '
 'Бакки. Всей командой пошли колумбийцы в атаку. Родригес играет в тело, возможно, своего будущего одноклубника. Несладко пришлось Марсело получить '
 'плечом в грудь от Хамеса. +5 минут. Если будут играть так, как се

...


100%|██████████| 729/729 [52:41<00:00,  4.34s/it]




1805
('«Магнитка» – ЦСКА. Коварж сравнял счет за 17 секунд до конца третьего периода и перевел игру в овертайм. Форвард «Магнитки» Ян Коварж сравнял счет '
 'за 17 секунд до окончания третьего периода третьего матча финальной серии Кубка Гагарина с ЦСКА (2:2). Чешскому игроку магнитогорцев помогли '
 'отличиться Данис Зарипов и Александр Семин. Таким образом, игра переходит в овертайм. Sports.ru ведет текстовую онлайн-трансляцию этого матча.\n'
 'Удаление у "Металлурга"! Батыршин за толчок клюшкой на пятаке у своих ворот. Тайм-аут взял Квартальнов. Все или ничего сейчас? гол!!!! Да Коста - '
 '2:3! Дубль!! 7-й гол в плей-офф! Как это было: кистевой бросок с левого края из верха круга вбрасывания оказался неберущимся для Кошечкина! И '
 'продолжает действовать правило финала плей-офф уже в третьем матче - забивший первым проигрывает... 2:3 в матче и 1-2 в финальной серии - ЦСКА '
 'вырывает победу в концовке первого овертайма! В состязание двух терпений и точных бросков сегодня были л

 75%|███████▌  | 3/4 [3:08:53<1:02:34, 3754.12s/it]

Epoch,Training Loss,Validation Loss,F1
1,0.488300,0.430454,0.862921
2,0.171800,0.121726,0.974155


100%|██████████| 4/4 [3:12:08<00:00, 2882.00s/it]

In [ ]:
m = AutoModelForSequenceClassification.from_pretrained("/content/checkpoint-108")
trainer = BaalTransformersTrainer(
    model=m,
    args=training_args,
    train_dataset=active_set,
    eval_dataset=test_set,
    tokenizer=None,
    compute_metrics=compute_metrics
)
output = trainer.evaluate()
output

{'eval_loss': 0.11491850763559341,
 'eval_f1': 0.9763779527559056,
 'eval_runtime': 9.8966,
 'eval_samples_per_second': 37.791,
 'eval_steps_per_second': 2.425}

In [ ]:
!cp -r "/content/checkpoint-108" "/content/drive/MyDrive/RuSportSum/bald"